In [121]:
import numpy as np
import scipy.stats as scps
from scipy.special.orthogonal import ps_roots
from scipy.interpolate import InterpolatedUnivariateSpline
%load_ext autoreload

%autoreload 2




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [122]:
from dc_egm import chop, polyline, aux_function, interpolate, upper_envelope
from ret import util, mutil, imutil, income, budget, mbudget, value_function, chpr, logsum

# Model parameters (default)

In [123]:
# Number of periods (fist period is t=1) 
Tbar = 25

# Number of grid points over assets
ngridm = 500

# Maximum level of assets
mmax = 50

# Number of quadrature points used in calculation of expectations
n_quad_points = 5

# Number of simulations
nsims = 10

# Interval of the initial wealth
init = [10, 30]

# Interest rate
r = 0.05

# Discount factor
beta = 0.95

# Standard deviation of log-normally distributed income shocks
sigma = 0.00

# Disutility of work
duw = 0.35

# CRRA coefficient (log utility if ==1)
theta = 1.95

# Careful with the coefficients here -- original code had the polynomial
# Coded as a + b * x - c * x**2 ... (note the crazy minus)
coeffs_age_poly = np.array([0.75, 0.04, -0.0002])

# Consumption floor (safety net in retirement)
cfloor = 0.001

# Scale of the EV taste shocks 
lambda_ = 2.2204e-16


In [124]:
# Initialize grids
quadstnorm = scps.norm.ppf(ps_roots(n_quad_points)[0])
quadw = ps_roots(n_quad_points)[1]

In [125]:
savingsgrid = np.linspace(0, mmax, ngridm)